In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import json
from functools import partial
import matplotlib.pyplot as plt
from pybpr import *

In [3]:
from pybpr import *
data_dir = '/home/rsandhu/zazzle/raw_data'

In [4]:
# tdf = pd.read_csv(os.path.join(data_dir, 'ml-10M100K', 'tags.dat'), sep='::', engine='python', header=None)
# tdf.columns = ['UserID','MovieID','Tag','Timestamp']
# tdf.head()
tdf_tag = pd.read_csv(os.path.join(data_dir, 'tag-genome','tags.dat'), sep='\t', header=None)
tdf_tag.columns = ['TagID','Tag','TagPopularity']
tdf_tag.head(100)
# tdf_movie = pd.read_csv(os.path.join(data_dir, 'tag-genome','movies.dat'), sep='\t', header=None)
# tdf_movie.columns = ['MovieID','Title','MoviePopularity']
# tdf_movie.head()

,TagID,Tag,TagPopularity
0,0,007,61
1,1,007 (series),24
2,2,18th century,37
3,3,1920s,42
4,4,1930s,55
...,...,...,...
95,95,bad,21
96,96,bad acting,182
97,97,bad cgi,34
98,98,bad ending,104


In [5]:
data_dir = '/home/rsandhu/zazzle/raw_data'
mdf = pd.read_csv(os.path.join(data_dir, 'ml-10M100K','movies.dat'), sep='::', engine='python', header=None)
mdf.columns = ['MovieID', 'Title','Genres']
#mdf.set_index('MovieID', inplace=True)
mdf.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
tdf = pd.read_csv(os.path.join(data_dir, 'tag-genome','tag_relevance.dat'), sep='\t', header=None)
tdf.columns = ['MovieID','TagID','Relevance']
tdf.drop(index=tdf.index[tdf.Relevance < 0.8], inplace=True)
tdf.head()

,MovieID,TagID,Relevance
28,1,28,0.894
62,1,62,0.979
63,1,63,0.987
185,1,185,0.947
202,1,202,0.898


In [7]:
rdf = pd.read_csv(os.path.join(data_dir, 'ml-10M100K', 'ratings.dat'), sep='::', engine='python', header=None)
rdf.columns= ['UserID','MovieID','Rating','Timestamp']
rdf.head()

,UserID,MovieID,Rating,Timestamp
0,1,122,5.0,838985046
1,1,185,5.0,838983525
2,1,231,5.0,838983392
3,1,292,5.0,838983421
4,1,316,5.0,838983392


In [8]:
# rdf = rdf[rdf.MovieID.isin(tdf.MovieID.unique())]
# tdf = tdf[tdf.MovieID.isin(rdf.MovieID.unique())]
# rdf.MovieID.nunique(), tdf.MovieID.nunique(), tdf.TagID.nunique()

In [9]:
ui = UserItemData(name='User-Metadata-only')
rdf = rdf[rdf.MovieID.isin(tdf.MovieID.unique())]
ui.add_positive_interactions(
     user_ids=rdf.UserID[rdf.Rating>=4.], 
     item_ids=rdf.MovieID[rdf.Rating>=4.]
)
ui.add_negative_interactions(
    user_ids=rdf.UserID[rdf.Rating<4.], 
    item_ids=rdf.MovieID[rdf.Rating<4.]
)
ui.add_user_features(
    user_ids=rdf.UserID.unique(), 
    feature_ids=rdf.UserID.unique()
)
ui.add_item_features(
     item_ids=tdf.MovieID, 
     feature_ids=tdf.TagID
)
ui

UserItemData(User-Metadata-only)
  Fuser     :( 69878× 69878) nnz=    69,878 (0.001%), empty rows/cols=     0/     0
  Fitem     :(  9542×  1096) nnz=   101,624 (0.972%), empty rows/cols=     0/     0
  Rpos      :( 69878×  9542) nnz= 4,986,252 (0.748%), empty rows/cols=    81/  1326
  Rneg      :( 69878×  9542) nnz= 4,940,884 (0.741%), empty rows/cols=    88/  1303

In [ ]:
ui.train_test_split(train_ratio_pos=0.8, train_ratio_neg=0.8, show_progress=True)

Train/Test Split (Cols):   9%|▉         | 120/1344 [00:12<02:07,  9.64it/s]

In [ ]:
adam = partial(
    torch.optim.Adam,
    lr=0.02,
    weight_decay=0.0
)
rs = RecSys(
    data = ui,
    model= HybridMF(ui.n_user_features, ui.n_item_features, n_latent=64),
    optimizer=partial(torch.optim.Adam, lr=0.05, weight_decay=0.0),
    output_dir='/kfs2/projects/zazzle/pybpr/notebooks/output/ml-run',
    loss_function=bpr_loss,
    log_level=2
)
rs

In [ ]:
rs.fit(n_iter=10, eval_every=1, batch_size=10000, eval_user_size=10000)

In [ ]:
rs.metrics